---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [364]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
#spam_data.head(10)

In [365]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [366]:
def answer_one():
    
    return spam_data.target.value_counts(normalize=True)[1]*100

In [367]:
#answer_one()

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [368]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    obj = CountVectorizer().fit(X_train)
    items = [i for i in obj.vocabulary_.keys()]
    ans = ''
    for i in items:
        if len(i)>len(ans):
            ans = i
    return ans

In [369]:
#answer_two()


### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [370]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    cv = CountVectorizer()
    X_train_vectorized = cv.fit_transform(X_train)
    X_test_vectorized = cv.transform(X_test)
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    y_pred = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, y_pred)

In [371]:
#answer_three()

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [426]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():

    vectorizer = TfidfVectorizer()
    X_train_transformed = vectorizer.fit_transform(X_train)
    feature_names = vectorizer.get_feature_names()
    data = vectorizer.idf_
    final_data = list(zip(feature_names, data))

    largest = sorted(final_data, key=lambda x: x[1], reverse=True)[:20]
    largest = sorted(largest, key=lambda x: x[0])
    largest = pd.Series([i[1] for i in largest], index=[i[0] for i in largest])


    smallest = sorted(final_data, key=lambda x: x[1])[:20]
    smallest = pd.Series([i[1] for i in smallest], index=[i[0] for i in smallest])

    return (smallest, largest)

In [407]:
#answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [374]:
def answer_five():
    vectorizer2 = TfidfVectorizer(min_df=3)
    X_train_transformed2 = vectorizer2.fit_transform(X_train)
    X_test_transformed = vectorizer2.transform(X_test)
    md = MultinomialNB(alpha=0.1).fit(X_train_transformed2, y_train)
    return roc_auc_score(y_test, md.predict(X_test_transformed))

In [375]:
#answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [376]:
def answer_six():
    
    temp = spam_data.copy()
    temp['length']=temp.text.str.len()
    return tuple([i for i in (temp.groupby('target').sum()/temp.groupby('target').count()).length.values])

In [377]:
#answer_six()

In [378]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [379]:
from sklearn.svm import SVC

def answer_seven():
    
    vectorizer3 = TfidfVectorizer(min_df=5)
    X_train_vectorised = vectorizer3.fit_transform(X_train)
    X_train_new = add_feature(X_train_vectorised, X_train.str.len())
    X_test_vectorised = vectorizer3.transform(X_test)
    X_test_new = add_feature(X_test_vectorised, X_test.str.len())
    md4 = SVC(C=10000)
    md4.fit(X_train_new, y_train)
    y_pred = md4.predict(X_test_new)
   
    return roc_auc_score(y_test, y_pred)

In [380]:
#answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [381]:
def answer_eight():
    
    temp2 = spam_data.copy()
    temp2['digits']=temp2.text.str.count(r'\d')
    return tuple([i for i in (temp2.groupby('target').sum()/temp2.groupby('target').count()).digits.values])


In [382]:
#answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [383]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    vectorizer4 = TfidfVectorizer(ngram_range=(1,3), min_df=5)
    X_train_vectorised2 = vectorizer4.fit_transform(X_train) 
    temp3 = add_feature(X_train_vectorised2, X_train.str.len())
    X_train_new2 = add_feature(temp3,X_train.str.count(r'\d'))
    X_test_vectorised2 = vectorizer4.transform(X_test)
    temp4 = add_feature(X_test_vectorised2, X_test.str.len())
    X_test_new2 = add_feature(temp4,X_test.str.count(r'\d'))
    md6 = LogisticRegression(C=100)
    md6.fit(X_train_new2, y_train)
    y_pred = md6.predict(X_test_new2)
    
    return roc_auc_score(y_test, y_pred)

In [384]:
#answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [418]:
def answer_ten():
    temp9 = spam_data.copy()
    temp9['non'] = temp.text.str.findall(r'(\W)').str.len()
    
    return tuple([float(i) for i in (temp9.groupby('target').sum()/temp9.groupby('target').count()).non.values])

In [421]:
#answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [387]:
def answer_eleven():
    
    count_vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb')
    
    X_train1 = count_vect.fit_transform(X_train)
    t1 = add_feature(X_train1, X_train.str.len())
    t2 = add_feature(t1, X_train.str.count(r'\d'))
    X_train_final = add_feature(t2, X_train.str.findall(r'\W').str.len())

    X_test1 = count_vect.transform(X_test)
    t1 = add_feature(X_test1, X_test.str.len())
    t2 = add_feature(t1, X_test.str.count(r'\d'))
    X_test_final = add_feature(t2, X_test.str.findall(r'\W').str.len())

    md11 = LogisticRegression(C=100)
    md11.fit(X_train_final, y_train)
    y_pred = md11.predict(X_test_final)
    score = roc_auc_score(y_test, y_pred)

    feature_names = np.array(count_vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = md11.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]]
    largest = feature_names[sorted_coef_index[:-11:-1]]

    
    return (score, pd.Series(smallest), pd.Series(largest))

In [388]:
#answer_eleven()